In [1]:
%tensorflow_version 2.x
import tensorflow as tf
import numpy as np
import os
import datetime

# Clear any logs from previous runs
!rm -rf ./logs/ 

TensorFlow 2.x selected.


In [2]:
mnist = tf.keras.datasets.fashion_mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train = np.expand_dims(x_train, axis=3)
y_train = np.expand_dims(y_train, axis=3)
x_test = np.expand_dims(x_test, axis=3)
y_test = np.expand_dims(y_test, axis=3)

x_train.shape, y_train.shape

4423680/4422102 [==============================] - 0s 0us/step


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.
  if __name__ == '__main__':


((60000, 28, 28, 1), (60000, 1))

In [9]:
"""
tf.random.set_seed(0)

activation_fn = tf.nn.relu #tf.nn.elu
n_filters = 128
n_kernals = 3


inputs = tf.keras.Input(shape=(28, 28, 1), name='img')
x = tf.keras.layers.Conv2D(n_filters, n_kernals, activation=activation_fn, padding='same')(inputs)
#x = tf.keras.layers.Conv2D(2, 1, activation=activation_fn, padding='same')(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(128, activation=activation_fn)(x)
x = tf.keras.layers.Dropout(0.2)(x)
predictions = tf.keras.layers.Dense(10, activation='softmax')(x)

model = tf.keras.Model(inputs, predictions, name='toy_resnet')
model.summary()
"""

tf.random.set_seed(0)

activation_fn = tf.nn.relu #tf.nn.elu
regularization_fn = tf.keras.regularizers.l2
n_filters = 256
n_kernals = 3
residual_tower_size = 2
regularization_rate=1e-4

inputs = tf.keras.Input(shape=(28, 28, 1), name='img')
x = tf.keras.layers.Conv2D(n_filters, n_kernals, kernel_regularizer=regularization_fn(regularization_rate), padding='same')(inputs)
x = tf.keras.layers.BatchNormalization()(x)
block_output = activation_fn(x)

# Build residual tower
for _ in range(residual_tower_size):
    x = tf.keras.layers.Conv2D(n_filters, n_kernals, kernel_regularizer=regularization_fn(regularization_rate), padding='same')(block_output)
    x = tf.keras.layers.BatchNormalization()(x)
    x = activation_fn(x)
    x = tf.keras.layers.Conv2D(n_filters, n_kernals, kernel_regularizer=regularization_fn(regularization_rate), padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.add([x, block_output])
    block_output = activation_fn(x)

x = tf.keras.layers.Conv2D(2, 1, kernel_regularizer=regularization_fn(regularization_rate), padding='same')(block_output)
x = tf.keras.layers.BatchNormalization()(x)
x = activation_fn(x)
x = tf.keras.layers.Flatten()(x)
#x = tf.keras.layers.Dense(256, kernel_regularizer=regularization_fn(regularization_rate))(x)
#x = tf.keras.layers.BatchNormalization()(x)
#x = activation_fn(x)
#x = tf.keras.layers.Dropout(0.5)(x)
predictions = tf.keras.layers.Dense(10, activation='softmax')(x)

model = tf.keras.Model(inputs, predictions, name='toy_resnet')
model.summary()

Model: "toy_resnet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
img (InputLayer)                [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
conv2d_10 (Conv2D)              (None, 28, 28, 256)  2560        img[0][0]                        
__________________________________________________________________________________________________
batch_normalization_10 (BatchNo (None, 28, 28, 256)  1024        conv2d_10[0][0]                  
__________________________________________________________________________________________________
tf_op_layer_Relu_10 (TensorFlow [(None, 28, 28, 256) 0           batch_normalization_10[0][0]     
_________________________________________________________________________________________

In [0]:
#checkpoint_dir = './training_checkpoints'
#checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
#checkpoint = tf.train.Checkpoint(optimizer=optimizer, model=model)
filepath = 'fashion_minst_ANN_model.h5'

In [0]:
#tf.random.set_seed(0)
#model = tf.keras.models.load_model(filepath)

In [0]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9) #tf.keras.optimizers.Adam()
model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [0]:
#manager = tf.contrib.checkpoint.CheckpointManager(checkpoint, directory="/tmp/model", max_to_keep=5)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                              patience=5, min_lr=0.00001)

log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

callback = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=8), 
            tf.keras.callbacks.ModelCheckpoint(
                filepath=filepath,
                save_best_only=True,
                monitor='val_loss',
                verbose=1),
            reduce_lr,
            tensorboard_callback]

In [14]:
model.fit(x_train, y_train, epochs=100, validation_split=0.2, callbacks=callback)

Train on 48000 samples, validate on 12000 samples
Epoch 1/100
47968/48000 [============================>.] - ETA: 0s - loss: 0.5382 - accuracy: 0.8446
Epoch 00001: val_loss improved from inf to 0.46090, saving model to fashion_minst_ANN_model.h5
48000/48000 [==============================] - 83s 2ms/sample - loss: 0.5380 - accuracy: 0.8447 - val_loss: 0.4609 - val_accuracy: 0.8688
Epoch 2/100
47968/48000 [============================>.] - ETA: 0s - loss: 0.4102 - accuracy: 0.8875
Epoch 00002: val_loss improved from 0.46090 to 0.40212, saving model to fashion_minst_ANN_model.h5
48000/48000 [==============================] - 82s 2ms/sample - loss: 0.4103 - accuracy: 0.8874 - val_loss: 0.4021 - val_accuracy: 0.8930
Epoch 3/100
47968/48000 [============================>.] - ETA: 0s - loss: 0.3650 - accuracy: 0.9025
Epoch 00003: val_loss improved from 0.40212 to 0.37902, saving model to fashion_minst_ANN_model.h5
48000/48000 [==============================] - 82s 2ms/sample - loss: 0.3650 -

In [0]:
#checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))
model = tf.keras.models.load_model(filepath)

In [16]:
model.evaluate(x_train,  y_train, verbose=2)

60000/60000 - 31s - loss: 0.2484 - accuracy: 0.9399


[0.2484153725941976, 0.93988335]

In [17]:
model.evaluate(x_test,  y_test, verbose=2)

10000/10000 - 5s - loss: 0.3419 - accuracy: 0.9081


[0.3418936897277832, 0.9081]

In [18]:
%tensorboard --logdir logs/fit

UsageError: Line magic function `%tensorboard` not found.
